In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
s = pd.read_csv('All_Stock.csv')
s['date'] = pd.to_datetime(s['date'])
s['mktcap'] = s['PRC'] * s['SHROUT']

In [84]:
ff = pd.read_csv('F-F_Research_Data_5_Factors_2x3_daily.CSV')
ff['date'] = pd.to_datetime(ff['Unnamed: 0'])
ff = ff.drop(columns= ['Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 0'])

In [4]:
table = pd.crosstab(s['date'], s['PERMNO'], values = s['mktcap'], aggfunc = sum)
companies = {}
for year in table.index:
    companies[year] = table.loc[year, :].sort_values(ascending = False).index[:100]

In [ ]:
years = list(companies.keys())
lesscomp = pd.DataFrame()
for i in tqdm(years):
    x = s[(s['date'] == i) & (s['PERMNO'].isin(list(companies[i])))]
    lesscomp = pd.concat([lesscomp, x])
s = lesscomp.copy()
s.to_csv('filtered_returns.csv', index=False)

In [ ]:
s = pd.read_csv('filtered_returns.csv')

In [27]:
s['spread'] = s['ASKHI'] - s['BIDLO']
s = s[pd.to_numeric(s['RET'], errors = 'coerce').notna()]
s['RET'] = s['RET'].apply(lambda x : float(x))
s = s[pd.to_numeric(s['RETX'], errors = 'coerce').notna()]
s['RETX'] = s['RETX'].apply(lambda x : float(x))
s['divfac'] = s['RET'] - s['RETX']
s['pctchgvol'] = s.groupby('PERMNO')['VOL'].pct_change()
s

,PERMNO,date,TICKER,BIDLO,ASKHI,PRC,VOL,RET,SHROUT,RETX,mktcap,spread,divfac
0,10401,1970-01-02,T,49.12500,49.87500,49.37500,79500.0,0.015424,549264.0,0.015424,2.711991e+07,0.75000,0.0
1,10604,1970-01-02,ARC,86.00000,90.00000,89.75000,27800.0,0.042090,44389.0,0.042090,3.983913e+06,4.00000,0.0
2,10786,1970-01-02,BS,26.87500,27.25000,27.12500,18000.0,0.000000,44022.0,0.000000,1.194097e+06,0.37500,0.0
3,10890,1970-01-02,BGH,157.25000,159.25000,158.75000,13600.0,0.008737,17232.0,0.008737,2.735580e+06,2.00000,0.0
4,11260,1970-01-02,C,34.25000,35.37500,35.25000,31400.0,0.025455,47942.0,0.025455,1.689956e+06,1.12500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286811,92602,2020-12-31,PM,81.29000,82.92000,82.79000,2892403.0,0.016078,1557316.0,0.016078,1.289302e+08,1.63000,0.0
1286812,92611,2020-12-31,V,216.20000,219.82001,218.73000,5922170.0,0.001694,1695677.0,0.001694,3.708954e+08,3.62001,0.0
1286813,92655,2020-12-31,UNH,344.04999,351.09000,350.67999,1969227.0,0.016493,948821.0,0.016493,3.327325e+08,7.04001,0.0
1286814,93002,2020-12-31,AVGO,433.04999,438.50000,437.85001,1015614.0,0.006945,407000.0,0.006945,1.782050e+08,5.45001,0.0


## Column names for 'S'

BIDLO = Bid price
ASKHI = Ask price
PRC = Price
VOL = Volume
RET = Returns
SHROUT = Shares outstanding
Spread = BID ASK spread
divfac = amount dividends effected returns

In [86]:
n = pd.DataFrame()
n['dailyreturns'] = s.groupby('date')['RET'].mean()
n['totalcap'] = s.groupby('date')['mktcap'].sum()
n['chngcap'] = n['totalcap'].pct_change()
n['divfac'] = s.groupby('date')['divfac'].mean()
n['spread'] = s.groupby('date')['spread'].mean()
n['shrout'] = s.groupby('date')['SHROUT'].mean()
n['SDT_RET'] = s.groupby('date')['RET'].std()
n = n.reset_index()
n['date'] = pd.to_datetime(n['date'])
n = pd.merge(n, ff, how = 'left', left_on = 'date', right_on = 'date')
stock = pd.read_csv('All_Stock.csv')
stock = stock[stock['TICKER'] == 'SPY'][['date', 'RET']]
stock['date'] = pd.to_datetime(stock['date'])
n = pd.merge(n, stock, how='left', left_on='date', right_on='date')
n = n[n['date'] >= '1978-10-03']
n = n.rename(columns= {'RET':'SPYRET'})
n['TMW_SPYRET'] = n['SPYRET'].shift(-1)
n = n[pd.to_numeric(n['TMW_SPYRET'], errors = 'coerce').notna()]
n['TMW_SPYRET'] = n['TMW_SPYRET'].apply(lambda x : float(x))
n['UP_DWN'] =  1 * (n['TMW_SPYRET'] > 0)
n = n[pd.to_numeric(n['TMW_SPYRET'], errors = 'coerce').notna()]
n['TMW_SPYRET'] = n['TMW_SPYRET'].apply(lambda x : float(x))
n['UP_DWN'] =  1 * (n['TMW_SPYRET'] > 0)
n

,date,dailyreturns,totalcap,chngcap,divfac,spread,shrout,SDT_RET,Mkt-RF,SMB,HML,RMW,CMA,RF,SPYRET,TMW_SPYRET,UP_DWN
2211,1978-10-03,-0.003019,4.421349e+08,-0.004191,0.000370,0.658750,95254.96,0.009717,-0.31,0.01,0.13,-0.35,-0.04,0.031,-0.017964,0.006098,1
2212,1978-10-04,0.004778,4.449798e+08,0.006434,0.000000,0.823750,95384.82,0.012843,0.38,-0.46,0.05,-0.06,0.00,0.031,0.006098,0.024242,1
2213,1978-10-05,0.002003,4.452071e+08,0.000511,0.000000,0.725000,95507.30,0.010809,0.22,0.19,0.08,0.10,-0.15,0.031,0.024242,0.047337,1
2214,1978-10-06,0.001968,4.461662e+08,0.002154,0.000060,0.665000,95507.30,0.009378,0.23,0.16,-0.07,0.09,-0.09,0.031,0.047337,0.107345,1
2215,1978-10-09,0.011510,4.516742e+08,0.012345,0.000000,0.778750,95254.96,0.011347,0.92,-0.37,-0.19,0.29,-0.02,0.031,0.107345,0.020408,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12860,2020-12-23,0.000843,2.395573e+10,-0.001125,0.000000,27.699945,1748866.63,0.015324,0.10,0.83,2.03,0.29,0.43,0.000,0.000899,0.003890,1
12861,2020-12-24,0.001969,2.401165e+10,0.002334,0.000104,44.291331,1748881.13,0.016186,0.21,-0.43,-0.18,0.23,-0.06,0.000,0.003890,0.008591,1
12862,2020-12-28,0.004419,2.428004e+10,0.011177,0.000000,45.152603,1762696.99,0.012615,0.46,-0.67,0.34,1.39,0.46,0.000,0.008591,-0.001908,0
12863,2020-12-29,0.001078,2.427635e+10,-0.000152,0.000000,33.126933,1768217.95,0.011280,-0.40,-1.42,0.23,0.75,-0.30,0.000,-0.001908,0.001427,1


# Columns for 'n' and comentary on importance

### this table is the aggragate for the top 100 firms in the market for each day

dailyreturns = average daily returns
    Previous days returns could be a large indicator in how the market moves
totalcap = total market capitalization for the 100 firms
    A changee in the total cap of the country could indicate volitility
changecap = percent change in totalcap
divfac = sum divfac for that day
    More dividends one day could put confidence pushing the returns of SPY up
spread = average spread
    Higher spread could indicate less trading and more conservatice ways to come
shrout = shares outstanding in the top 100 firms
    This could indicate macro changes from reports and splits to effect psycology
Mkt-Rf = market returns - risk free
    total market returns not that would be outside of our firms investigates
SMB = smallcap minus largecap
    If small caps react different than large then there could be macro effects
HML = value minus growth companies
    Same as SMB
RMW = robust minus weak
    Similar to SMB
CMA = conservative minus agressive
    Similar to CMA
RF = risk free
    Incase needed, but changing treasury rates could effect the economic outlook
SPYRET = SPY returns for the day
TMW_SPYRET = the SPY returns for the next day
UP_DWN = 1 -> Market up tomorrow, 0 -> Market down tomorrow
    This is the target variable
STD_RET = standard deviation of daily returns
    more volitiltiy could effect the next day of trading.


In [87]:
n.to_csv('daily_aggragate_data.csv', index=False)

In [3]:
r = pd.read_csv('daily_aggragate_data.csv')

In [5]:
## Creating the 5 day lag of returns

r['dailyreturns_1'] = r['dailyreturns'].shift(1)
r['dailyreturns_2'] = r['dailyreturns'].shift(2)
r['dailyreturns_3'] = r['dailyreturns'].shift(3)
r['dailyreturns_4'] = r['dailyreturns'].shift(3)

In [6]:
# Averaged of the past 5 days of reuturns

r['pastreturns'] = (r['dailyreturns'] + r['dailyreturns_1'] + r['dailyreturns_2'] + r['dailyreturns_3'] + r['dailyreturns_4'])/5

In [7]:
## Square of returns

r['squaredreturns'] = r['dailyreturns']**2
r['date'] = pd.to_datetime(r['date'])

In [8]:
t = pd.read_csv('treasureis.csv')
t['CALDT'] = pd.to_datetime(t['CALDT'])
t['TDRATE'] = pd.to_numeric(t['TDRATE']).fillna(0)
t = t.groupby('CALDT').sum().reset_index()
t = t[['CALDT', 'TDRATE']]

In [68]:
r = pd.merge(r, t, how = 'left', left_on= 'date', right_on= 'CALDT')

In [65]:
## Getting measurs of the economy through banking activity, this is hoping to be able to look for and account for Macro Economic effects

d = pd.read_csv('deposits.csv')
d['date'] = pd.to_datetime(d['RSSD9999'], format='%Y%m%d')
d = d.rename(columns={'RCFD0391': 'Bank Investment', 'RCFD2170': 'Bank Total Assets', 'RCFD2200': 'Bank Deposits'})
d = d[['date', 'Bank Investment', 'Bank Total Assets', 'Bank Deposits']]
d = d.groupby('date').sum().reset_index()
d['Bank Deposits'] = d['Bank Deposits'].pct_change()
d['Bank Total Assets'] = d['Bank Total Assets'].pct_change()
d['Bank Investment'] = d['Bank Investment'].pct_change()
d['quarter'] = d['date'].dt.quarter
d['year'] = d['date'].dt.year

,date,Bank Investment,Bank Total Assets,Bank Deposits,quarter,year
0,1978-03-31,NaN,NaN,NaN,1,1978
1,1978-06-30,NaN,0.064776,0.053329,2,1978
2,1978-09-30,NaN,-0.091406,-0.093299,3,1978
3,1978-12-31,NaN,0.205243,0.194443,4,1978
4,1979-03-31,NaN,-0.037152,-0.035188,1,1979
...,...,...,...,...,...,...
164,2019-03-31,NaN,0.010688,-0.003986,1,2019
165,2019-06-30,NaN,0.006593,0.026759,2,2019
166,2019-09-30,NaN,0.009729,-0.012861,3,2019
167,2019-12-31,NaN,0.005566,0.067226,4,2019


In [66]:
r['quarter'] = r['date'].dt.quarter
r['year'] = r['date'].dt.year

In [72]:
r = pd.merge(r, d, how = 'left', left_on = ['quarter', 'year'], right_on= ['quarter', 'year'] )

In [73]:
r['Bank Investment'] = r['Bank Investment'].fillna(0)

In [75]:
r.to_csv('newdata.csv', index = False)

### Once we determined that many of our models did not work properly we decided to look elsewhere and this is where we wrangled this data.

# New Stuff starts here

In [4]:
d = pd.read_csv('filtered_returns.csv')

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
import numpy as np

In [14]:
## We needed to coerce the returns data to be numeric so we can work with it.

## Returns data from WRDS/ CRSP is not always 100% clean

d = d[pd.to_numeric(d['RET'], errors = 'coerce').notna()]
d['RET'] = d['RET'].map(lambda x: float(x))

/var/folders/z5/fdb0bxz54sq5wvl409z8r5980000gn/T/ipykernel_45203/3330655457.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['RET'] = d['RET'].map(lambda x: float(x))


### This starts the area where we look at the percentiles of the returns of the market to gain insight to more normalized data

In [15]:
p = np.percentile(d['RET'], 50) # return 50th percentile, e.g median.
print(p)

0.0


### Creating the [ercentilr sin  steps of 5 to add the the dataframe

In [35]:
df = pd.DataFrame()
for i in range(5,100,5):
    df['X' + str(i)] = d.groupby('date')['RET'].quantile(i/100)



In [36]:
df

,X5,X10,X15,X20,X25,X30,X35,X40,X45,X50,X55,X60,X65,X70,X75,X80,X85,X90,X95
date,,,,,,,,,,,,,,,,,,,
1970-01-02,-0.014116,-0.008894,-0.006411,-0.005027,-0.001283,0.000000,0.000000,0.001419,0.004119,0.006478,0.010485,0.014363,0.017391,0.020165,0.023597,0.026247,0.028259,0.032650,0.040665
1970-01-05,-0.018168,-0.015349,-0.011234,-0.007993,-0.005693,-0.003909,-0.002416,-0.001515,0.000000,0.003724,0.004772,0.005764,0.006402,0.007399,0.009664,0.012536,0.019770,0.028463,0.036234
1970-01-06,-0.029552,-0.023593,-0.019602,-0.017976,-0.015306,-0.013921,-0.012788,-0.010730,-0.009143,-0.007861,-0.007038,-0.006727,-0.004465,-0.003318,-0.001488,0.000000,0.003718,0.007956,0.010265
1970-01-07,-0.024214,-0.019163,-0.015416,-0.013264,-0.011659,-0.009249,-0.007048,-0.005562,-0.003995,-0.002740,0.000000,0.000000,0.000777,0.003082,0.004349,0.005617,0.007535,0.013729,0.018216
1970-01-08,-0.017060,-0.012885,-0.011541,-0.008439,-0.007331,-0.005258,-0.002886,0.000000,0.000836,0.002771,0.003563,0.004794,0.005791,0.007063,0.008053,0.009578,0.011377,0.017836,0.026207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,-0.010462,-0.005240,-0.003967,-0.002976,-0.002048,-0.001032,-0.000298,0.001381,0.002521,0.003425,0.003819,0.004234,0.004842,0.006083,0.006912,0.007937,0.010267,0.011410,0.016761
2020-12-28,-0.012080,-0.006379,-0.004905,-0.002554,-0.000983,-0.000520,0.000843,0.001668,0.002964,0.004482,0.005807,0.006430,0.007072,0.008633,0.009715,0.011287,0.013402,0.017880,0.023810
2020-12-29,-0.010727,-0.009144,-0.007525,-0.006383,-0.006050,-0.005001,-0.003902,-0.002754,-0.002111,-0.001828,0.000024,0.001805,0.003594,0.004519,0.005253,0.006338,0.007910,0.011631,0.020131


In [38]:
a = pd.read_csv('daily_aggragate_data.csv')

In [39]:
a

,date,dailyreturns,totalcap,chngcap,divfac,spread,shrout,SDT_RET,Mkt-RF,SMB,HML,RMW,CMA,RF,SPYRET,TMW_SPYRET,UP_DWN
0,1978-10-03,-0.003019,4.421349e+08,-0.004191,0.000370,0.658750,95254.96,0.009717,-0.31,0.01,0.13,-0.35,-0.04,0.031,-0.017964,0.006098,1
1,1978-10-04,0.004778,4.449798e+08,0.006434,0.000000,0.823750,95384.82,0.012843,0.38,-0.46,0.05,-0.06,0.00,0.031,0.006098,0.024242,1
2,1978-10-05,0.002003,4.452071e+08,0.000511,0.000000,0.725000,95507.30,0.010809,0.22,0.19,0.08,0.10,-0.15,0.031,0.024242,0.047337,1
3,1978-10-06,0.001968,4.461662e+08,0.002154,0.000060,0.665000,95507.30,0.009378,0.23,0.16,-0.07,0.09,-0.09,0.031,0.047337,0.107345,1
4,1978-10-09,0.011510,4.516742e+08,0.012345,0.000000,0.778750,95254.96,0.011347,0.92,-0.37,-0.19,0.29,-0.02,0.031,0.107345,0.020408,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9237,2020-12-23,0.000843,2.395573e+10,-0.001125,0.000000,27.699945,1748866.63,0.015324,0.10,0.83,2.03,0.29,0.43,0.000,0.000899,0.003890,1
9238,2020-12-24,0.001969,2.401165e+10,0.002334,0.000104,44.291331,1748881.13,0.016186,0.21,-0.43,-0.18,0.23,-0.06,0.000,0.003890,0.008591,1
9239,2020-12-28,0.004419,2.428004e+10,0.011177,0.000000,45.152603,1762696.99,0.012615,0.46,-0.67,0.34,1.39,0.46,0.000,0.008591,-0.001908,0
9240,2020-12-29,0.001078,2.427635e+10,-0.000152,0.000000,33.126933,1768217.95,0.011280,-0.40,-1.42,0.23,0.75,-0.30,0.000,-0.001908,0.001427,1


### Merging the percentils with other variables that we created above to get other market factors and to use their predictive ablity.

In [41]:
df = pd.merge(a, df, how = 'left', left_on = 'date', right_index = True)

In [44]:
import math

In [96]:
df['SINTMWSPYRET'] = df['TMW_SPYRET'].map(lambda x: math.sin(x * 100))

### Experimenting wiht the results using the price being normalized to sin to keep it in a better range.

In [99]:
spy = pd.read_csv('spyret.csv').dropna()

In [100]:
spy = spy.rename(columns = {'BIDLO':'LOW', 'ASKHI':'HIGH','PRC':'PRICE','BID':'CLOSINGBID','ASK':'CLOSEINGASK'})

In [101]:
spy['date'] = pd.to_datetime(spy['date'])
spy = spy[['date','LOW','HIGH','PRICE','CLOSINGBID','CLOSEINGASK','VOL']]
spy

,date,LOW,HIGH,PRICE,CLOSINGBID,CLOSEINGASK,VOL
5965,1993-02-02,44.12500,44.37500,44.34375,44.31250,44.34375,201300.0
5966,1993-02-03,44.37500,44.84375,44.81250,44.81250,44.84375,529400.0
5967,1993-02-04,44.87500,45.09375,45.00000,44.96875,45.00000,531500.0
5968,1993-02-05,44.71875,45.06250,44.96875,44.96875,45.00000,492100.0
5969,1993-02-08,44.90625,45.12500,44.96875,44.84375,44.87500,596100.0
...,...,...,...,...,...,...,...
12991,2020-12-24,367.45001,369.03000,369.00000,368.97000,368.98999,26457853.0
12992,2020-12-28,371.07001,372.59000,372.17001,372.13000,372.14001,39000402.0
12993,2020-12-29,370.82999,374.00000,371.45999,371.47000,371.47000,53680451.0
12994,2020-12-30,371.57001,373.10001,371.98999,371.92001,371.92999,49455259.0


In [102]:
df['date'] = pd.to_datetime(df['date'])

## merging all of the data we could want with in this data frame

In [103]:
df = pd.merge(spy, df, how = 'left', on = 'date')

In [104]:
df

,date,LOW_x,HIGH_x,PRICE_x,CLOSINGBID_x,CLOSEINGASK_x,VOL_x,LOW_y,HIGH_y,PRICE_y,...,X55,X60,X65,X70,X75,X80,X85,X90,X95,SINTMWSPYRET
0,1993-02-02,44.12500,44.37500,44.34375,44.31250,44.34375,201300.0,44.12500,44.37500,44.34375,...,0.003362,0.005391,0.008108,0.009342,0.011046,0.012827,0.014375,0.018219,0.021659,0.870934
1,1993-02-03,44.37500,44.84375,44.81250,44.81250,44.84375,529400.0,44.37500,44.84375,44.81250,...,0.011098,0.012585,0.013621,0.015758,0.018100,0.020447,0.021907,0.027264,0.040422,0.406299
2,1993-02-04,44.87500,45.09375,45.00000,44.96875,45.00000,531500.0,44.87500,45.09375,45.00000,...,0.007182,0.009021,0.010642,0.012585,0.014750,0.018742,0.021051,0.025016,0.032828,-0.069344
3,1993-02-05,44.71875,45.06250,44.96875,44.96875,45.00000,492100.0,44.71875,45.06250,44.96875,...,0.000000,0.002073,0.003073,0.005113,0.008612,0.010919,0.013699,0.015543,0.026057,0.000000
4,1993-02-08,44.90625,45.12500,44.96875,44.84375,44.87500,596100.0,44.90625,45.12500,44.96875,...,-0.003113,-0.002140,0.000000,0.000000,0.003278,0.004249,0.010087,0.014019,0.017157,-0.640309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7008,2020-12-24,367.45001,369.03000,369.00000,368.97000,368.98999,26457853.0,367.45001,369.03000,369.00000,...,0.003819,0.004234,0.004842,0.006083,0.006912,0.007937,0.010267,0.011410,0.016761,0.757255
7009,2020-12-28,371.07001,372.59000,372.17001,372.13000,372.14001,39000402.0,371.07001,372.59000,372.17001,...,0.005807,0.006430,0.007072,0.008633,0.009715,0.011287,0.013402,0.017880,0.023810,-0.189644
7010,2020-12-29,370.82999,374.00000,371.45999,371.47000,371.47000,53680451.0,370.82999,374.00000,371.45999,...,0.000024,0.001805,0.003594,0.004519,0.005253,0.006338,0.007910,0.011631,0.020131,0.142216
7011,2020-12-30,371.57001,373.10001,371.98999,371.92001,371.92999,49455259.0,371.57001,373.10001,371.98999,...,0.001542,0.002260,0.002947,0.005485,0.007773,0.009025,0.011140,0.015764,0.021914,0.486518


In [111]:
len(df[df['TMW_SPYRET'] >= 0])/len(df)

0.5476971338941965

In [112]:
df.to_csv('dataframe.csv', index = False)